In [ ]:
from kgfw.extract.doc_handle import BaseDocxHandle

document = BaseDocxHandle('pdftoword.docx')
data = document.get_docx_structure()
# catalog = document.get_catalog()
# document.recovery_docx()

In [ ]:
data

In [ ]:
from kg_extract.doc_handle import BasePptxHandle
document = BasePptxHandle('test.pptx')
data = document.get_pptx_structure()

In [ ]:
import pandas as pd

data = pd.read_excel('dublin2229_GPIO_Pinmux_PA1_upadte.xlsx')

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
# es_ctrl = Elasticsearch([{'host':'localhost','port':9200}])
es_ctrl = Elasticsearch([{'host':'10.166.152.49','port':9200}], proxies = {'http': '45.76.191.105:19004'})

In [ ]:
import re
import json
data = es_ctrl.get(index='trouble-shooting-task', id='9jZFRXYBvGheiu9GCm8o')
# re.sub('\"topic\": (.*?)', '',json.dumps(data))

In [ ]:
update_data = {'doc':{'username': 'executor',
     'password': '123456',
     'authority': 'normal',
     'groups': [
        {
             'project': 'Common',
             'role': 'visitor'
         },
         {
             'project': 'TroubleShooting',
             'role': 'executor'
         }
     ]}}

es_ctrl.update(index='admin', id='jwbx83UBOPmBSJMOKjDC', body=update_data)

In [ ]:
pages = [{
    'project': 'Common',
    'pages':[{'name':'pinmux','role':['visitor'],'href':'#/pinmux'},
#              {'name':'hunter','role':['visitor'],'href':'#/hunter'},
             {'name':'feedback','role':['visitor'],'href':'#/feedback'},
#              {'name':'learning','role':['visitor'],'href':'#/learning'},
            ]
    },
    {
    'project': 'Babel',
    'pages':[
             #{'name':'data','role':['visitor'],'href':'#/data'},
             {'name':'symbols','role':['visitor'],'href':'#/symbols'},
             {'name':'viewer','role':['visitor'],'href':'#/viewer'}
            ]       
    },
    {
    'project': 'TroubleShooting',
    'pages':[{'name':'edit','role':['administrator','executor','visitor'],'href':'#/trouble_shooting_template_edit'},
            {'name':'task','role':['administrator','executor','visitor'],'href':'#/trouble_shooting_task'}
            ]       
    }
]

for elm in pages:
    _ = es_ctrl.index(index='pages', body=elm)

In [ ]:
es_ctrl.search(index='admin')

In [ ]:
doc = {
    "mappings":{
        "properties":{
            "username":{
                "type":"keyword"
            },
            "car_number":{
                "type":"keyword"
            },
            "state":{
                "type":"keyword"
            },
        }
    }
}
es_ctrl.indices.create(index='stop-car-data',body =doc)

In [ ]:
doc = {
    "mappings":{
        "properties":{
            "data_source_name":{
                "type":"keyword"
            }
        }
    }
}
es_ctrl.indices.create(index='babel-datas',body =doc)

In [ ]:
_ = es_ctrl.index(index='stop-car-account', body={'username': '133923','password': '123456', 'cookies':'pass'})

In [ ]:
es_ctrl.indices.delete(index='pages')

In [ ]:
def query_with(kv):
    doc = {
            "query": {
                "bool": {
                    "must": []
                }
            }
        }
    for k, v in kv:
        doc["query"]["bool"]["must"].append({"term": {k: v}})
    return doc

es_ctrl.search(index='stop-car-data', size=200, body=query_with([['username', '133923']]))['hits']['hits']

In [ ]:
doc = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "template_id": "ojvTSnYBChHa9MqEsktD"
                    }
                }
            ]
        }
    }
}

es_ctrl.search(index='trouble-shooting-task', body=doc)

In [ ]:
es_ctrl.delete(index='babel-viewers', id='hjjNFXwBGRmkQQYJK83E')

In [ ]:
data = es_ctrl.search(index='babel-datas')['hits']['hits']
data

In [ ]:
from io import BytesIO
import pandas as pd

def mind_export_to_csv(node_data, node=None):
    temp = []
    node = node if node else ''
    if type(node_data) == list:
        for elm in node_data:
            if elm.__contains__('children'):
                temp.extend(mind_export_to_csv(elm['children'], node+'/'+elm['topic']))
            else:
                temp.append([elm['topic'], node, elm['Status'], elm['Executor']])
    else:
        if node_data.__contains__('children'):
            temp.extend(mind_export_to_csv(node_data['children'], node+'/'+node_data['topic']))
        else:
            temp.append([node_data['topic'], node, node_data['Status'], node_data['Executor']])
    return temp

res = es_ctrl.get(index='trouble-shooting-task', id='6Tu7S3YBChHa9MqEu0za')
data = res['_source']['nodeData']['children']
temp = mind_export_to_csv(data)
temp = pd.DataFrame(temp, columns=['Tasks', 'Responsible', 'Achieved', 'Executor'])

# excel_file = BytesIO()
xlwriter = pd.ExcelWriter('demo.xlsx', engine='xlsxwriter')
temp.to_excel(xlwriter, res['_source']['TemplateName'])
# xlwriter.save()
# excel_file.seek(0)

In [ ]:
workbook = xlwriter.book
worksheet = xlwriter.sheets[res['_source']['TemplateName']]

format_l = workbook.add_format()
format_l.set_pattern(1)
format_l.set_bg_color('gray')
worksheet.write('B1:D2', 'Ray', format_l)
xlwriter.save()

In [ ]:
worksheet.write?

In [ ]:
#coding: utf-8
import xlsxwriter
 
# 创建工作簿
file_name = "first_book.xlsx"
workbook = xlsxwriter.Workbook(file_name)
 
# 创建工作表
worksheet = workbook.add_worksheet('sheet1')
 
# 写单元格
worksheet.write(0, 0, 'id')
worksheet.write(0,1, 'name')
worksheet.write(0,2, 'class')
worksheet.write(0,3, 'data')
 
# 写行
worksheet.write_row(1, 0, [1, 2, 3])
 
# 写列,其中列D需要大写
worksheet.write_column('D2', ['a', 'b', 'c'])
 
# 关闭工作簿
workbook.close()

In [ ]:
import requests


response = requests.get("http://127.0.0.1:5010/get/")

In [ ]:
response.content

In [ ]:
import pandas as pd

data = pd.read_csv('test.csv', encoding='gbk')
data.iloc[:,[0]].values

In [ ]:
import base64

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = 'administrator'.encode('utf-8')
key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)

file_out = open("encrypted.bin", "wb")
[ file_out.write(x) for x in (cipher.nonce, tag, ciphertext) ]
file_out.close()

In [ ]:
file_in = open("encrypted.bin", "rb")
nonce, tag, ciphertext = [ file_in.read(x) for x in (16, 16, -1) ]

# let's assume that the key is somehow available again
cipher = AES.new(key, AES.MODE_EAX, nonce)
data = cipher.decrypt_and_verify(ciphertext, tag)
data

In [ ]:
from Crypto.PublicKey import RSA

key = RSA.generate(1024)
private_key = key.export_key()
file_out = open("private.pem", "wb")
file_out.write(private_key)
file_out.close()

public_key = key.publickey().export_key()
file_out = open("receiver.pem", "wb")
file_out.write(public_key)
file_out.close()

In [ ]:
cipher.block_size

In [ ]:
import requests

In [ ]:
import gzipimport base64
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Cipher import PKCS1_v1_5

ciphertext = 'mzlvr1MNY6y2k3TZvh3DdoGyrNu7xy2AfUXSTaT+Gma+ceZo2Cn4Nixep2dIkeyHW8RKj1JNgZr5eGn8ErgRtPzYx2+tvBCt1pXuZK80LGvfc3ajDbiwd1FkuJvnRcyPrayaD8jfG55EJv0RL/P2+bkyco3cNCAtR2A+Ys7/GMA='
private_key = RSA.import_key(open("private.pem").read())


# Decrypt the session key with the private RSA key
cipher_rsa = PKCS1_v1_5.new(private_key)
session_key = cipher_rsa.decrypt(base64.b64decode(ciphertext),"ERROR")

# # Decrypt the data with the AES session key
# cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
# data = cipher_aes.decrypt_and_verify(ciphertext, tag)
# print(data.decode("utf-8"))

In [ ]:
session_key.decode()

In [ ]:
a = "{\"isNewReport\":1,\"userId\":\"\",\"carNo\":\"甘-J2L199\",\"privacyOption\":1,\"longitude\":103.97679,\"latitude\":30.67841,\"distance\":3000,\"openId\":\"\",\"signType\":\"MD5\",\"timestamp\":1609574891332,\"nonce\":\"35053231609574891332\",\"sign\":\"DA8FEA8741ACBA2F7B7B8A2BDE994F8D\"}"
b = "{\"isNewReport\":1,\"userId\":\"c36b86825b004d648bfbbbc1b58a675e\",\"carNo\":\"川-C3D682\",\"privacyOption\":1,\"longitude\":0,\"latitude\":0,\"distance\":3000,\"openId\":\"\",\"signType\":\"MD5\",\"timestamp\":1609739888281,\"nonce\":\"75341421609739888281\",\"sign\":\"48DF5B8598ADEC7BE0D1B90D268C227E\"}"

In [ ]:
import gzip
import json

with open('IWD.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)

In [ ]:
len(gzip.compress(json.dumps(json_data).encode()))

In [ ]:
len(json.dumps(json_data))

In [ ]:
import requests

url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/Shared%20Documents/Forms/AllItems.aspx"
headers={
    'Host': "ericsson.sharepoint.com",
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
    'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'Accept-Language': "en-US,en;q=0.5",
    'Accept-Encoding': "gzip, deflate, br",
    'Referer': "https://login.microsoftonline.com/",
    'Upgrade-Insecure-Requests': "1",
    'Connection': "keep-alive",
    'Cookie': "rtFa=WSBaz0JfgJmPAlBlLH9kCuaTD118ctqDxr0kXGnh8iwmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjU2MjAxMzE5NzUzNjgwOSMyMjRBQTU5Ri1EMDRCLUIwMDAtQzk0Mi00QzVDRUQzQUNGMUYozAClMgzoD5Fb1dM6x/qPug79a5y4Vm4RVuDMFNgRBSJ+AmS6ksdUyyrfQTr1aWxmsYQ9dIyTWb4LxNO+lZy6z54d3i7rvJ2fPYGRXbmWUKyjyg54ts1B7NKL31q0OSLTrmnt6XdSKoSaUy2X1znQG9oVPpWNfMvmxUWOScpOluNYaHv+3iY14kvLGJKZPZ1jVbG4JDDI1T1VEzAFkaK2aSQHuEfBr5TUj7d0mm1u2D6Ak6hlH6k1hsuGmiM4MU8/n//yQhfjrB0Ujw4InleEjQSGX/QCb/KOUeTULRHP18+JVeprkQn0II/rcUvWx6iTz3qne4p8Y2MwXCjJhaiDfQAAAA==; FedAuth=77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjgsMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAbGl2ZS5jb20sMCMuZnxtZW1iZXJzaGlwfGRlbmdqdW4ubGVpQGVyaWNzc29uLmNvbSwxMzI1NjIwMTI5NTAwMDAwMDAsMTMyNTIzODM2MDAwMDAwMDAwLDEzMjU2Mjg3NzE5NzM4MDMwOSwxMDEuMzMuMTI3LjExNSw2Niw5MmU4NGNlYi1mYmZkLTQ3YWItYmU1Mi0wODBjNmI4Nzk1M2YsLDRiOWYxYzUyLTdiYzktNGQyZC1iYzMxLTI4ZDBlYmU2OTNhOSwyMmEyODc5ZC02ODdlLTQ0N2YtODBiZC1mYmM4YTU0NDdiOWQsMjJhMjg3OWQtNjg3ZS00NDdmLTgwYmQtZmJjOGE1NDQ3YjlkLCwwLDEzMjU2MjA0OTE5NjkxMTk2NywxMzI1NjQ2MDUxOTY5MTE5NjcsLCxleUo0YlhOZlkyTWlPaUpiWENKRFVERmNJbDBpZlE9PSwyNjUwNDY3NzQzOTk5OTk5OTk5LDEzMjU2MjAxMzE4MDAwMDAwMCxlYzhmOTFhNS01NDY1LTQ2NmYtYjMwOS00NGQxZDNkMGUyODAsSUhuNWlPWDNMNXlhazRsRHpXMi9kY01GVmgyMGZmbzNxUlpYRkU4emZaZGxZdEJYUjBWTjV5OW80SFFRVDN5cDRuRk9HK3pJcVNwV0dmU09CZEI3Q2QrcWF5eUM0cU84WTB3Tm51cWYvNktRVC9nRzk4ZEN0enpGZHo2Tm5qSUY3Z2JLenZoK1paVFpVRVFEZUZ4UVBaZUxNcXA4cTVqQkZOOTRqZFErMXJLZElETHpzcFJzSmxEbnZYSStlOE5ZUzc5ZWxrYU9iSUp2bWp6cStSWVYwUnpJQ1E3OHZNam9FQU9CVzR4MnFLMVpmd29vVmZQbDBWMWtsRmVoSElPc1B0NlZnd2dmSVljcFNteVdGVHZiSXFoamxPUkZKT2xKbG9VREI3K0d5Y2I0UmNyUEpxOGZZSUxmNVBhR3YrZ041R2VhMlNuNWtUenE2SXJGQkZwRHZ3PT08L1NQPg==; CCSInfo=MS8zMC8yMDIxIDg6MTM6MzcgQU1Dq8P8FwjF7qoLaotIfXAD6pzzfAC8Igyur4WzdR48fhkTNrX4T+U+3N3OGnqnnUrgXhcg4TBb1y4oj4ElQoVwUSiMNMYySW9/trH+ncjNMtm1KXp8MzcwHnTeA2pyw0mwz11zlQAr0cuoo0sfcVUiPj2hFTZIVN90z2pDmmAL62MFZEANBNPMo0/ul6PgfNsMo4JMggLqUR4JWdV4ePmD11m0t+U2o7G5r/xeXyuYLLQvNdsl2BWtCCBeB3DbMUMJz+yOzqmXFnInWBUwy6dyAgZXupBc2D+ukeXWcP/6SKyKiP47iZHOsQRx5dV4KSNa9sJXfUMs4fK1eZH5+lnrFAAAAA==; isso=dHJ1ZSwyODUwMCwyODgwMCwxNjExNzU2MjkxNDk2LDE2MTE3NTY1OTE0OTY="
}
payload={"parameters":{"__metadata":{"type":"SP.RenderListDataParameters"},"RenderOptions":5707527,"AllowMultipleValueFilterForTaxonomyFields":True,"AddRequiredFields":True}}
cookies={
    'CCSInfo': "My8yMC8yMDIxIDI6NTQ6NDkgQU0SMXTuAaPn+TowrIe//27HqthHXxWJjYizZHGKbMmdXtXbmBIDu1aRZBYw8ipimYkwQLvJKNVC0OZ55+psjEE13zU7CPz8E7Chl+RLRL2rJTB4Yv5rbdJYvVfbb/PnFG8nORIq4KNgIzVOr+dCed6iynlpKOxRinH9aJHMhVsT9TvDrw4iDCl4JDVPLAPKBoOiKlk7f15QWAi4riJW7xqongVMQzhVX8YFG6U7nSh0DGpv8H/H9HMkKeHRPd+nsj5gNoaCoxm3kILZPYDJ9lAj6C/Z2R2adoiIIxTMF14UAlwMpWp9tzVFydm29ndBsOyOlNQnIU3o30rJhXWpFrTkFAAAAA==",
    'FedAuth': "77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjksMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAbGl2ZS5jb20sMCMuZnxtZW1iZXJzaGlwfGRlbmdqdW4ubGVpQGVyaWNzc29uLmNvbSwxMzI2MDQxNTc2NDAwMDAwMDAsMTMyNjAyNjM0MzcwMDAwMDAwLDEzMjYwNTAyMTkxNTQ3NDE0Nyw0NS43Ni4xOTEuMTA1LDY2LDkyZTg0Y2ViLWZiZmQtNDdhYi1iZTUyLTA4MGM2Yjg3OTUzZiwsYmRmODgwMzktZWZmZi00MjQwLTg3NjAtMTVlNmVlNWNkNTlkLDM4YWIyMDUyLTlkY2QtNDBiNy1hZmMxLTI4NmRlNzBhZTY3YiwzOGFiMjA1Mi05ZGNkLTQwYjctYWZjMS0yODZkZTcwYWU2N2IsLDAsMTMyNjA0MTkzOTE0MjI0NjY1LDEzMjYwNjc0OTkxNDIyNDY2NSwsLGV5SjRiWE5mWTJNaU9pSmJYQ0pEVURGY0lsMGlmUT09LDI2NTA0Njc3NDM5OTk5OTk5OTksMTMyNjA0MTU3OTAwMDAwMDAwLGVjOGY5MWE1LTU0NjUtNDY2Zi1iMzA5LTQ0ZDFkM2QwZTI4MCwsLCwsdEVrNUt0eGN0WVFHaDhMRWlMeENpUlFpcDlnS2xpc294SDEwbngra245bVNLQ0FzSnF2b2N3aXplODY0eUE3OXExOTNUd2cweFJJbk82QXFEdXlVdjBud00xd3VhVk1Xbkp6b3NMdXVORmlZbkNyR2Nsd0ZySjhaQXRLODFKK21qTng4K1ZkVnNyVW1PcE1SYkptNko1TEpoT1NPSFc1cHBBdXNkRkc3QlV6bkpDQ0NSaW9ueGZITy9ENmdTSVVCakh5WWYrTGNXbFJMU0RLWFFOVUVWcm5iUVVzZE5LVkoyL2piaGxkZDFoYkI5dGdkNC9zcDZUcHRlZ1UyM3huWVEwNEkrbmI2cWhlK0tWNUFEeXdTeXRtaDhhdzFyK0F3TjdwN3dTUEFoeDMvNkEzL2k2YmJKaVV6VnkyMUtFT2FDQ3poemlGUzVnNVRaZmRDWVNyWW13PT08L1NQPg==",
    'isso':"VHJ1ZSwyODUwMCwyODgwMCwxNjE1OTcxMDg4NTE3LDE2MTU5NzEzODg1MTc=",
    'rtFa':"/yXlAC5/seA8TZzf/fOHNn8ATL7LkjkP9WzZnGPY3igmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjYwNDE1NzkxNTYzMDYzOCM1RUZEQjQ5Ri1GMDBGLUIwMDAtQzk0Mi00QzU2Njk1OUM4RUQEDYYDoopu/0XVTR/XYuZ5hzqxqlSpImF3YVKwFbGFkoNlKmTq5fqYYqvFb6BZ6Cdub+Iu3/sXIgqR60lX2Z5qmDwl6rLntzeUmgV+bLuA17F454o1n28Wu4gvIeB20X4LOdBCVoPzcQ8idJUDox8JwCyJB8np2IOciuZzM/uDYHwywtWzq08h6BuuPk0HQ3uVsdqPi8cSWE3N7A+267CEmDuxFgxUZAYBZgH5i0+q/bJHLIeQOXuFaEkR2keUXvH2jYTckUGq2EW6fnUAZSr5bktSeGr0ZhKJJaXSSnos7ofgorCb7XF2L/DV3hvcMYkwJ4hRF9F6SdLmHreo0lI6fQAAAA==",
}

In [ ]:
import requests

request = requests.session()
ret=request.get(url,headers=headers,cookies=cookies)
# ret.content

In [ ]:
url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/_api/web/GetListUsingPath(DecodedUrl=@a1)/RenderListDataAsStream?@a1='/sites/PDURadioChengduFirmware/Shared Documents'&View=1d04f153-2aa3-4553-b6e2-14b66655e6d4&TryNewExperienceSingle=TRUE"
ret1=request.post(url,cookies=ret.cookies.get_dict())

In [ ]:
url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/_api/web/GetListUsingPath(DecodedUrl=@a1)/RenderListDataAsStream?@a1='/sites/PDURadioChengduFirmware/Shared Documents'&RootFolder=/sites/PDURadioChengduFirmware/Shared Documents/2018 TeamBuilding&View=1d04f153-2aa3-4553-b6e2-14b66655e6d4&TryNewExperienceSingle=TRUE"
ret2=request.post(url,cookies=ret1.cookies.get_dict())

In [ ]:
import json
json.loads(ret2.content)

In [ ]:
temp = "/sites/PDURadioChengduFirmware/Shared Documents/2018 TeamBuilding/2018_TB_GroupPhotos.rar"
temp[1:]

In [ ]:
url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/_layouts/15/download.aspx?UniqueId=0cf748e8%2D85b9%2D4b8a%2D9b9e%2Dda6ecf75d039"
ret3=request.post(url)

In [ ]:
with open("testtest.docx", "wb") as f:
    f.write(ret3.content)

In [ ]:
import gzip
import base64

result = gzip.compress(ret3.content)
base64_str = base64.b64encode(result)


In [ ]:
len(base64_str)

In [ ]:
from kgfw.extract.doc_handle import BaseDocxHandle

document = BaseDocxHandle(BytesIO(bytes(ret3.content)))
data = document.get_docx_structure()

In [ ]:
data = {'headers':headers, 'cookies':cookies}
with open("sharepoint_config.json","w") as f:
    json.dump(data,f)
    
# with open("sharepoint_config.json",'r') as load_f:
#     load_dict = json.load(load_f)
# load_dict

In [ ]:
import pdfplumber

with pdfplumber.open("test.pdf") as pdf:
    first_page = pdf.pages[1]
    print(first_page.extract_text())

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)
model.eval()

In [ ]:
text = "General Description"
encoded_input = tokenizer(text, return_tensors='pt')

In [ ]:
encoded_input

In [ ]:
input_ids = torch.tensor(tokenizer.encode("General Description")).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
# last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

In [ ]:
hidden_states = outputs[2]
token_vecs = hidden_states[-2][0]
# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
sentence_embedding

In [ ]:
len(hidden_states[-2][0])

In [ ]:
import numpy as np  
from milvus import Milvus, MetricType  

# 初始化一个Milvus类，以后所有的操作都是通过milvus来的  
milvus = Milvus(host='localhost', port='19530')  

# 向量个数  
num_vec = 5000  
# 向量维度  
vec_dim = 768  
# name  
collection_name = "test_collection"  
# 创建collection，可理解为mongo的collection  
collection_param = {  
    'collection_name': collection_name,  
    'dimension': vec_dim,  
    'index_file_size': 32,  
    'metric_type': MetricType.IP  # 使用内积作为度量值  
}  
milvus.create_collection(collection_param)  

# 随机生成一批向量数据  
# 支持ndarray，也支持list  
vectors_array = np.random.rand(num_vec, vec_dim)  

# 把向量添加到刚才建立的collection中  
status, ids = milvus.insert(collection_name=collection_name, records=vectors_array)  # 返回 状态和这一组向量的ID  
milvus.flush([collection_name])  

# 输出统计信息  
print(milvus.get_collection_stats(collection_name))  

# 创建查询向量  
query_vec_array = np.random.rand(1, vec_dim)  
# 进行查询,  
status, results = milvus.search(collection_name=collection_name, query_records=query_vec_array, top_k=5)  
print(status)  
print(results)  

# 如果不用可以删掉  
status = milvus.drop_collection(collection_name)  

# 断开、关闭连接  
milvus.close()

In [ ]:
import json
import requests

con = []
for i in range(0,10):
    con.append("hello world!"+str(i))
data = {'text': con}
url = 'http://localhost:8000/extract/text_embedding'
res = requests.post(url,data=data)

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

class BaseBertEmbedding(object):

    def __init__(self, model):
        self.tokenizer = BertTokenizer.from_pretrained(model)
        self.model = BertModel.from_pretrained(model)

    def get_input_ids(self, text):
        return torch.tensor(self.tokenizer.encode(text)).unsqueeze(0)  # Batch size 1

    def get_last_hidden_states(self, text):
        return self.model(self.get_input_ids(text))[0]
    
text_embedding = BaseBertEmbedding('bert-base-uncased')

In [ ]:
from urllib3 import encode_multipart_formdata

url = 'http://localhost:8000/extract/extract_docx'
data = {}
headers = {}
data['upload_file'] = ("test.docx", open("test.docx", 'rb').read())
encode_data = encode_multipart_formdata(data)
data = encode_data[0]
headers['Content-Type'] = encode_data[1]
res = requests.post(url, headers=headers, data=data)

In [ ]:
from io import BytesIO
from kgfw.extract.doc_handle import BaseDocxHandle

with open('test.docx', 'rb') as f:
    file = BytesIO(f.read())
    # f = open("testtest.docx", 'rb').read()
document = BaseDocxHandle(file)
data = document.get_docx_structure()
data

In [ ]:
import requests

url="http://123.57.202.1:8000/vip"
query_headers={
    'Host': "123.57.202.1:8000",
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
    'Accept': "application/json, text/plain, */*",
    'Accept-Language': "en-US,en;q=0.9",
    'Accept-Encoding': "gzip, deflate",
    'Referer': "http://123.57.202.1:8000/",
    'Connection': "keep-alive",
    'Cookie': "session=eyJhdXRoIjp0cnVlfQ.YGE35w.x0slYTLo31jxg6VY1s9B2Lihjd4"
}

In [ ]:
request = requests.session()
ret=request.get(url,headers=query_headers)

In [ ]:
import json
json.loads(ret.content)

In [ ]:
len(es_ctrl.search(index='stop-car-data', body=query_with([['username', str(666666)]]),size=100)['hits']['hits'])

In [ ]:
def query_with(kv):
    doc = {
            "query": {
                "bool": {
                    "must": []
                }
            }
        }
    for k, v in kv:
        doc["query"]["bool"]["must"].append({"term": {k: v}})
    return doc

res = es_ctrl.search(index='stop-car-data', body=query_with([['username', str(133923)]]))['hits']['hits']
tmp = []
for elm in res:
    tmp.append(elm['_source'])
tmp

In [ ]:
es_ctrl.search(index='stop-car-data', body=query_with([['username', str(133923)]]))['hits']['hits']

In [ ]:
try:
    data = json.loads(ret.content)
    for car in data['vips']:
        elm = {'username': '133923', 'car_number':car['card'], 'info':car}
        _ = es_ctrl.index(index='stop-car-data', body=elm)
except Exception as e:
    traceback.print_exc()

In [ ]:
add_url = "http://123.57.202.1:8000/add"

add_headers={
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Content-Length": "21",
    "Content-Type": "application/json;charset=UTF-8",
    "Cookie": "session=eyJhdXRoIjp0cnVlfQ.YGE35w.x0slYTLo31jxg6VY1s9B2Lihjd4",
    "Host": "123.57.202.1:8000",
    "Origin": "http://123.57.202.1:8000",
    "Referer": "http://123.57.202.1:8000/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36",
}
payload = {"cards": "川AU2779"}
ret1=request.post(add_url, data=json.dumps(payload), headers=add_headers)

In [ ]:
if ret1.content.decode() == 'ok':
    print('------')

In [ ]:
import json
data = {'query_url':"http://123.57.202.1:8000/vip",'query_headers':query_headers,'add_url':"http://123.57.202.1:8000/add",'add_headers':add_headers}
with open("config.json","w") as f:
    json.dump(data,f)

In [ ]:
import json
import pandas as pd

tmp = pd.read_excel('stop_car.xlsx')
tmp

In [ ]:
json.loads(tmp.to_json(orient='records'))[0]

In [ ]:
import datetime
datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
from apscheduler.schedulers.background import BackgroundScheduler

def test():
    print('-------')
    
def test1():
    print('!!!!')
    
scheduler = BackgroundScheduler()
scheduler.add_job(test, 'interval', minutes=0.05)
scheduler.add_job(test1, 'interval', minutes=0.1)
scheduler.start()

In [ ]:
scheduler.pause()

In [ ]:
scheduler.resume()

In [ ]:
num = es_ctrl.count(index='stop-car-data')['count']
num

In [ ]:
import threading,time

def run(n):
    for i in range(0,5):
        print("task  ",i)
        time.sleep(1)

start_time=time.time()
t1=threading.Thread(target=run,args=("t1",))

In [ ]:
t1.start()

In [ ]:
import redis

# config="121.41.42.251:6379:1".split(':')
config="localhost:6379:1".split(':')
pool = redis.ConnectionPool(host=config[0], port=config[1], decode_responses=True, db=config[2])
redis_con = redis.Redis(connection_pool=pool)

#### 

In [ ]:
import json

In [ ]:
with open('..\\test\\v2ray.json', 'r') as f:
    tmp = json.loads(f.read())

In [ ]:
tmp